# Tutorial: Advection Equation with DeepONet neural operator

In this tutorial we will show how to solve a neural operator problem using `DeepONet`. Specifically, ....

We import the relevant modules first.

In [62]:
import matplotlib.pyplot as plt
from pina.geometry import CartesianDomain
from pina import LabelTensor, Trainer, Condition
from pina.equation import Equation
from pina.problem import TimeDependentProblem, SpatialProblem
import torch


## Generating the data

We first generate the data we will use for training.

In [85]:
# fixing mu ~ U[0.05, 1], and sigma=0.0002
r1 = 0.05
r2 = 1.
mu = (r1 - r2) * torch.rand(1) + r2
sigma = torch.tensor([0.0002])

# initial condition
def initial_condition(pts):
    return torch.exp(-(pts - mu)**2 / sigma) * (sigma * torch.pi) ** (-0.5)

def solution(pts, t):
    return torch.stack([pts, t.repeat(pts.shape, 1), initial_condition(pts - t)]).T

def compute_solution(pts, times):
    # fixing mu ~ U[0.05, 1], and sigma=0.0002
    mu = (0.05 - 1) * torch.rand(1) + 1
    sigma = torch.tensor([0.0002])
    # compute solution for each timestep
    solutions = torch.stack([solution(pts, t) for t in times])
    
    return solutions


In [94]:
t = torch.linspace(0, 1, 100)
x = torch.linspace(0, 2, 256)
solutions = LabelTensor(torch.stack([compute_solution(x, t) for _ in range(1000)]), ['x', 't', 'u'])

In [112]:
solutions.extract(['u']).shape

torch.Size([1000, 100, 256, 1])

In [ ]:
class AdvectionProblem(SpatialProblem, TimeDependentProblem):

    output_variables = ['u']
    input_variables = ['x', 't']

    # define initial condition
    def initial_condition(input_, output_):
        u_expected = -torch.sin(torch.pi*input_.extract(['x']))
        return output_.extract(['u']) - u_expected

    # assign output/ spatial and temporal variables
    output_variables = ['u']
    spatial_domain = CartesianDomain({'x': [-1, 1]})
    temporal_domain = CartesianDomain({'t': [0, 1]})

    # problem condition statement
    conditions = {
        'data': Condition(input_pts = ),
    }